# Usage
We start by exploring the data-processing pipeline part of `DAMAST`.
We consider a manufactured dataset of Automatic Identification System (AIS) messages.
The data is generated for 150 boats, where the minimal length of a trajectory is 30 messages, and the maximal length is 1000

In [ ]:
!pip install damast

import polars
import damast.domains.maritime.ais.data_generator as generator

data = generator.AISTestData(number_of_trajectories=1000, min_length=25, max_length=300)

The data is stored in a [polars.LazyFrame](https://docs.pola.rs/api/python/stable/reference/lazyframe/index.html), and we can inspect the first and last 5 messages in the dataset.

In [ ]:
print(data.dataframe)

The dataset consists of 11 columns, which we will go through in detail.

## Data-specification
The Maritime Mobile Service Identity (MMSI) used to identify a ship. It *should* be a 9 digit number whose first integer should be between 2 and 7.
The data we have generated should contain some invalid numbers. Let us inspect these.

In [ ]:
from damast.domains.maritime.data_specification import MMSI
df = data.dataframe
invalid_mmsis = df.filter((polars.col('mmsi') < MMSI.min_value) | (polars.col('mmsi') > MMSI.max_value))
invalid_mmsis

Before sending this data to a machine learning algorithm, one would have to filter out invalid data.
We can do this by creating a `damast.core.DataSpecification` describing what valid output we would like in our data-frame.

In [ ]:
from damast.core import DataSpecification, MinMax
mmsi_spec = DataSpecification(name="mmsi", description="Maritime Mobile Service Identity", representation_type=int,
                              value_range=MinMax(MMSI.min_value, MMSI.max_value))

We have here described what data this column is supposed to describe, how the data is represented in Python, and its minimum and maximum range.
Next, we create a `damast.core.MetaData` object that we can apply to the dataframe.

In [ ]:
from damast.core import MetaData,ValidationMode
metadata = MetaData([mmsi_spec])
metadata.apply(df.lazy(), ValidationMode.UPDATE_DATA)

Of course, we do not want to do this process manually per row. Therefore, we can create a `DataSpecification` per row, and let the `damast.core.AnnotatedDataFrame` handle the validation of the data. We can choose between three ways of handling the input data with metadata, we can either use:
- `ValidationMode.READONLY`: Reads in the data, checks it against the meta-data and throws an error if the data does not adhere to the data-specification.
- `ValidationMode.UPDATE_METADATA`: Update the metadata based on the input in the annotated data-frame. This might change the representation type, column name and valid rages of the data.
- `ValidationMode.UPDATE_DATA`: Update data so that it adheres to the meta-data.

In [ ]:
from damast.core.metadata import DataCategory
from damast.core.dataframe import AnnotatedDataFrame
dataspec = {
    "annotations": {"comment": "This is a autogenerated test data set"},
    "columns": [
        {"name": "mmsi", "is_optional": False, "category": DataCategory.STATIC,
         "value_range":{"MinMax": {"min": MMSI.min_value, "max": MMSI.max_value}}},
        {"name": "lon", "is_optional": False, "unit": "deg", "category": DataCategory.DYNAMIC},
        {"name": "lat", "is_optional": False, "unit": "deg", "category": DataCategory.DYNAMIC},
        {"name": "date_time_utc", "is_optional": False, "category": DataCategory.DYNAMIC},
        {"name": "sog", "is_optional": False, "category": DataCategory.DYNAMIC},
        {"name": "cog", "is_optional": False, "category": DataCategory.DYNAMIC},
        {"name": "true_heading", "is_optional": False, "category": DataCategory.DYNAMIC},
        {"name": "nav_status", "is_optional": False, "category": DataCategory.DYNAMIC},
        {"name": "rot", "is_optional": False, "category": DataCategory.DYNAMIC},
        {"name": "message_nr", "is_optional": False, "category": DataCategory.DYNAMIC},
        {"name": "source", "is_optional": False, "category": DataCategory.DYNAMIC},
    ]
}
metadata = MetaData.from_dict(dataspec)
data = generator.AISTestData(number_of_trajectories=1000, min_length=25, max_length=300)
adf = AnnotatedDataFrame(data.dataframe, metadata, validation_mode=ValidationMode.UPDATE_DATA)
adf

## Data-processing
Say we want to repeat this process on any data-set we read in. Then, we should create a `damast.core.dataprocessing.DataProcessingPipeline`.
A pipeline consists of pipeline-elements, that is a set of transformations on the original dataset.
We start by creating a Pipeline-element that drops all rows missing an `"mmsi"` entry.

In [ ]:
from damast.data_handling.transformers.filters import DropMissingOrNan
from damast.core.dataprocessing import DataProcessingPipeline
pipeline = DataProcessingPipeline(name="Remove missing MMSI columns",
                                  base_dir="./output_dir",
                                  inplace_transformation=True)
pipeline.add(name="Remove MMSI column",
             transformer=DropMissingOrNan(),
             name_mappings={"x": "mmsi"})

transformed_adf = pipeline.transform(adf)
transformed_adf

In [ ]:
transformed_adf.collect()

### Data Pipelines with multiple input sources
When multiple input sources exist and should be merged, a join operator (transformer) can be designed.
The join can, but must not necessarily involve two pipelines as illustrated in the following.
The pipeline is named using the 'data_source' argument, but this is optiona


In [ ]:
import damast
from damast.core.transformations import PipelineElement

class JoinByTime(PipelineElement):                                                                                                                                                                             
    @damast.core.describe("Join data by timestamp")                                                                                        
    @damast.core.input({                                                                                                          
                           "timestamp": {},                                                                                       
                           "lon": {},                                                                                             
                           "lat": {},                                                                                             
                       })                                                                                                         
    @damast.core.input({                                                                                                          
                            "timestamp": {},                                                                                      
                            "lat": {},                                                                                            
                            "lon": {}                                                                                             
                        }, label='other'                                                                                          
    )                                                                                                                             
    @damast.core.output({})                                                                                                       
    def transform(self, df: AnnotatedDataFrame, other: AnnotatedDataFrame) -> AnnotatedDataFrame:                                 
        other_timestamp = self.get_name('timestamp', datasource='other')                                                          
        df_timestamp = self.get_name('timestamp')                                                                                 
                                                                                                                                  
        df._dataframe = df.join(other._dataframe, left_on=df_timestamp, right_on=other_timestamp)                                 
        return df

event_time = data.dataframe.drop_nulls().select(polars.col('date_time_utc')).item(0,0)
events_dataframe = polars.from_dict({'latitude': [40.0, 40.1], 'longitude': [10.0,10.2], 'timestamp': [event_time, event_time], 'event_type': ["accident", "accident"]})
events_metadata = AnnotatedDataFrame.infer_annotation(events_dataframe)
events_adf = AnnotatedDataFrame(events_dataframe, metadata=events_metadata)

In [ ]:
from damast.data_handling.transformers.cycle_transformer import CycleTransformer

events_pipeline = DataProcessingPipeline(name="events",
                                  base_dir="./output_dir") \
    .add("lat_cycle_transform", CycleTransformer(n=180), name_mappings={'x': 'latitude'}) \
    .add("lon_cycle_transform", CycleTransformer(n=90), name_mappings={'x': 'longitude'}) \

pipeline = DataProcessingPipeline(name="ais_events_merge",
                                  base_dir="./output_dir") \
    .add("lat_cycle_transform", CycleTransformer(n=180), name_mappings={'x': 'lat'}) \
    .add("lon_cycle_transform", CycleTransformer(n=90), name_mappings={'x': 'lon'}) \
    .join("events", data_source=events_pipeline, operator=JoinByTime(),
              name_mappings = {
                  'df': {
                      "timestamp": "date_time_utc",
                  },
                  'other': {
                      "timestamp": "timestamp",
                      "lon": "longitude",
                      "lat": "latitude"
                  }
              },
    )


To run the pipeline, all required datasource aka inputs need to be provided as arguments.
While the default input is 'df', the datasource for the 'join' operator requires to be provided via
 the keyword of the same name, here 'events'

In [ ]:
joined_adf = pipeline.transform(df=adf, events=events_adf)

In [ ]:
joined_adf.head(10)

In [ ]:
joined_adf.head(10).collect()